In [1]:
import Lexer.Lexer as lx
import re

In [2]:
default_depth = 500
max_depth = default_depth

class SyntaxBuilder:
    def __init__(self,path_grammar, init_symbol = 'S'):
        self.path_grammar = path_grammar
        
        self.init_symbol = init_symbol
        self.grammar = {}
        self.non_terminals = set()
        
        self.first = {}
        
        self.following = {}
        self.explored = set() #set to keep state of following
        
        self.predictions = {}
        
    def loadGrammar(self):
        f = open(self.path_grammar)
        lines = f.readlines()
        f.close()
        
        for line in lines:
            line = line.strip().split()
            if line[0] not in self.grammar:
                self.non_terminals.add(line[0])
                self.first[line[0]] = set()
                self.following[line[0]] = set()
                self.predictions[line[0]] = []
                self.grammar[line[0]] = []
            self.grammar[line[0]].append(line[1:])
        
        self.following[self.init_symbol] = {'$'} # Add to first symbol
        
    def primeros(self, v, precalc = False):
        global max_depth
        max_depth-=1
        
        if max_depth <=0 or len(v)==0:
            max_depth+=1
            return {'e'}
        
        if len(v) == 1 and v[0]=='e':
            max_depth+=1
            return {'e'}
        
        if v[0] not in self.non_terminals:
            max_depth+=1
            return {v[0]}
        
        if len(v) == 1 and v[0] in self.non_terminals:
            if precalc: return self.first[v[0]] # Used when we have already calculated it for non-terminals
            
            productions = self.grammar[v[0]] 
            first = set()
            for p in productions:
                first |= self.primeros(p)
            max_depth+=1
            self.first[v[0]] |= first
            return first
        
        first = self.primeros([v[0]])
        
        if 'e' in first:
            if len(v)>1:
                first.discard('e')
                first |= self.primeros(v[1:])
        max_depth+=1
        return first
    
    
    def siguiente(self, non_terminal): # S is the non-terminal
        global max_depth, default_depth
        
        self.explored.add(non_terminal)
        
        for production in self.grammar[non_terminal]:
            for i in range(len(production)):
                p = production[i]
                if p in self.non_terminals:
                    if p not in self.explored:
                        self.siguiente(p)
                    
                    max_depth = default_depth
                    first = self.primeros(production[i+1:])
                    
                    self.following[p] |= first - {'e'}
                    if 'e' in first:
                        self.following[p].add(non_terminal)
    def predict(self, S, prod):
        first = self.primeros(prod, True)
        if 'e' in first:
            first.discard('e')
            return first | self.following[S]
        else:
            return first
        
    def calcFirsts(self):
        global max_depth, default_depth
        for S in self.non_terminals:
            max_depth = default_depth
            self.primeros([S])
    
    def calcFollowing(self):
        
        for non_terminal in self.non_terminals:
            if non_terminal not in self.explored:
                self.siguiente(self.init_symbol)
        
        added = True #Placeholder, does nothing
        while added:
            added = False
            for non_terminal in self.non_terminals:
                current = self.following[non_terminal].copy()
                for element in self.following[non_terminal]:
                    if element in self.non_terminals:
                        to_add = self.following[element]
                        added = True
                        current |= to_add
                        current -= {non_terminal}
                        current -= {element}
                self.following[non_terminal] = current
    
    def calcPredictions(self):
        for k,productions in self.grammar.items():
            for production in productions:
                self.predictions[k].append(list(self.predict(k,production)))
    
    def calculateAll(self):
        self.calcFirsts()
        self.calcFollowing()
        self.calcPredictions()

In [3]:
file_path = 'input.txt'
grammar_path = 'grammar.txt'

def mainExists(lexer):
    
    pass

def main():
    lexer = lx.Lexer(file_path)
    lexer.readFile()
    
    grammar = SyntaxBuilder(grammar_path,'S')
    grammar.loadGrammar()
    grammar.calculateAll()
    

<resource,3,1>
<id,aswap,3,10>
<tk_par_izq,3,16>
<tk_par_der,3,17>
<var,5,5>
<id,c,5,9>
<tk_asig,5,11>
<id,chars,5,14>
<tk_par_izq,5,20>
<tk_cadena,"abcde",5,21>
<tk_par_der,5,28>
<var,6,5>
<id,d,6,9>
<tk_asig,6,11>
<id,chars,6,14>
<tk_par_izq,6,20>
<tk_cadena,"zyxwv",6,21>
<tk_par_der,6,28>
<write,7,5>
<tk_par_izq,7,11>
<id,c,7,12>
<tk_coma,7,13>
<id,d,7,15>
<tk_par_der,7,16>
<id,c,8,5>
<tk_swap,8,7>
<id,d,8,11>
<write,9,5>
<tk_par_izq,9,11>
<id,c,9,12>
<tk_coma,9,13>
<id,d,9,15>
<tk_par_der,9,16>
<id,d,10,5>
<tk_swap,10,7>
<id,c,10,11>
<write,11,5>
<tk_par_izq,11,11>
<id,c,11,12>
<tk_coma,11,13>
<id,d,11,15>
<tk_par_der,11,16>
<write,12,5>
<tk_par_izq,12,11>
<id,c,12,12>
<tk_swap,12,14>
<id,d,12,18>
<tk_par_der,12,19>
<procedure,14,5>
<id,adump,14,15>
<tk_par_izq,14,21>
<id,x,14,22>
<tk_cor_izq,14,23>
<tk_multi,14,24>
<tk_coma,14,25>
<tk_multi,14,26>
<tk_cor_der,14,27>
<tk_dos_puntos,14,28>
<int,14,29>
<tk_par_der,14,32>
<fa,15,5>
<id,i,15,8>
<tk_asig,15,9>
<tk_num,1,15,12>
<to,15,14